In [1]:
import paddle.fluid as fluid
import paddle
import numpy as np
from paddle.fluid.debugger import draw_block_graphviz

## Shared params

In [2]:
dataset = [
    [np.random.random(size=(128, 3, 10, 10)).astype('float32'), np.ones((128,1)).astype('float32')],
    [np.random.random(size=(128, 3, 24, 24)).astype('float32'), np.ones((128,1)).astype('float32')],
    [np.random.random(size=(128, 3, 48, 48)).astype('float32'), np.ones((128,1)).astype('float32')],
]

In [3]:
# Name all param_attr and bias_attr to share params
def shared_net(image):
    y = fluid.layers.conv2d(name="conv2d", input=image, num_filters=3, filter_size=3, act='relu', param_attr='conv2d_w', bias_attr='conv2d_b')
    y = fluid.layers.pool2d(name="pool2d", input=y, pool_size=y.shape[2:])
    y = fluid.layers.fc(name='predict', input=y, size=1, param_attr='fc_w', bias_attr='fc_b')
    return y

In [4]:
def train_net(name, shape):
    # Different program for diffent input size
    program = fluid.Program()
    with fluid.program_guard(program):
        label = fluid.layers.data(name='label' + name, shape=[1], dtype='float32')
        image = fluid.layers.data(name='image' + name, shape=shape, dtype='float32')
        predict = shared_net(image)
        infer_program = program.clone(for_test=True)
        cost = fluid.layers.square_error_cost(input=predict, label=label)
        avg_cost = fluid.layers.mean(cost)
        return image, label, predict, avg_cost, program, infer_program

In [5]:
# Different program and optimizer for diffent input size
image1, label1, predict1, avg_cost1, train_program1, infer_program1 = train_net("1", [3,10,10])
image2, label2, predict2, avg_cost2, train_program2, infer_program2 = train_net("2", [3,24,24])
image3, label3, predict3, avg_cost3, train_program3, infer_program3 = train_net("3", [3,48,48])

# And with different optimizer
optimizer = fluid.optimizer.Adam(learning_rate=0.001)
_ = optimizer.minimize(avg_cost1)
optimizer2 = fluid.optimizer.Adam(learning_rate=0.001)
_ = optimizer2.minimize(avg_cost2)
optimizer3 = fluid.optimizer.Adam(learning_rate=0.001)
_ = optimizer3.minimize(avg_cost3)

In [6]:
draw_block_graphviz(avg_cost1.block, path="g1.pdf")
draw_block_graphviz(avg_cost2.block, path="g2.pdf")

In [7]:
exe = fluid.Executor(fluid.CPUPlace())
exe.run(fluid.default_startup_program())

[]

In [8]:
for i in range(1000):
    c1 = exe.run(program=train_program1, fetch_list=[avg_cost1], feed={
            image1.name: dataset[0][0], label1.name: dataset[0][1]
    })
    c2 = exe.run(program=train_program2, fetch_list=[avg_cost2], feed={
            image2.name: dataset[1][0], label2.name: dataset[1][1]
    })
    c3 = exe.run(program=train_program3, fetch_list=[avg_cost3], feed={
            image3.name: dataset[2][0], label3.name: dataset[2][1]
    })
    if i % 100 == 0:
        print(i, c1[0][0], c2[0][0], c3[0][0])

(0, 4.21338, 6.256412, 7.2955246)
(100, 0.12623334, 0.032384813, 0.11959931)
(200, 0.1316835, 0.027014297, 0.103084795)
(300, 0.12968615, 0.024039526, 0.093308955)
(400, 0.12740204, 0.02158181, 0.08514366)
(500, 0.124925956, 0.019551685, 0.07851617)
(600, 0.12276098, 0.017818484, 0.072931156)
(700, 0.12098904, 0.016324164, 0.06812219)
(800, 0.11956181, 0.015047586, 0.06395402)
(900, 0.118354075, 0.013962875, 0.060271367)


In [9]:
fluid.io.save_params(exe, dirname="shared")
!ls shared

conv2d.b_1  conv2d.w_1	predict.b_1  predict.w_1


In [10]:
# For another inputsize, we can reuse the shared network
img_size = 129
image, label, predict, avg_cost, train_program1, infer_program1 = train_net("new_129", [3,img_size,img_size])
result, avg_cost = exe.run(program=train_program1, fetch_list=[predict, avg_cost], feed={
    image.name: np.random.random((8, 3, img_size, img_size)).astype('float32'),
    label.name: np.ones((8, 1)).astype('float32')
})
print(avg_cost)

[0.17281255]


In [11]:
result

array([[1.3968543],
       [1.3360865],
       [1.4340291],
       [1.3299111],
       [1.3533896],
       [1.4627941],
       [1.3668158],
       [1.5841353]], dtype=float32)